In [1]:
import os
import time
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, text

load_dotenv()
start = time.time()

# Connection string
conn_str = (
    f"postgresql://{os.getenv('PG_USER')}:"
    f"{os.getenv('PG_PASSWORD')}@"
    f"{os.getenv('PG_HOST')}:"
    f"{os.getenv('PG_PORT')}/"
    f"{os.getenv('PG_DATABASE')}"
)

engine = create_engine(conn_str, connect_args={"connect_timeout": 5})

# ---- Run SQL ----
with engine.connect() as conn:
    # Ping
    conn.execute(text("SELECT 1"))
    print("DB ping OK")

    df_tables = pd.read_sql("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name
    """, conn)

df_tables

DB ping OK


,table_name
0,album
1,artist
2,customer
3,employee
4,genre
5,invoice
6,invoice_line
7,media_type
8,playlist
9,playlist_track


In [2]:
with engine.connect() as conn:
    df_cols = pd.read_sql("""
        SELECT table_name, column_name
        FROM information_schema.columns
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position
    """, conn)

for table, group in df_cols.groupby("table_name"):
    print(table)
    print("  " + ", ".join(group["column_name"]))

print("took", round(time.time() - start, 2), "s")

album
  album_id, title, artist_id
artist
  artist_id, name
customer
  customer_id, first_name, last_name, company, address, city, state, country, postal_code, phone, fax, email, support_rep_id
employee
  employee_id, last_name, first_name, title, reports_to, birth_date, hire_date, address, city, state, country, postal_code, phone, fax, email
genre
  genre_id, name
invoice
  invoice_id, customer_id, invoice_date, billing_address, billing_city, billing_state, billing_country, billing_postal_code, total
invoice_line
  invoice_line_id, invoice_id, track_id, unit_price, quantity
media_type
  media_type_id, name
playlist
  playlist_id, name
playlist_track
  playlist_id, track_id
track
  track_id, name, album_id, media_type_id, genre_id, composer, milliseconds, bytes, unit_price
took 0.92 s


In [3]:
query1 = """
WITH jazz_customers AS (
  SELECT DISTINCT i.customer_id
  FROM invoice i
  JOIN invoice_line il ON il.invoice_id = i.invoice_id
  JOIN track t ON t.track_id = il.track_id
  JOIN genre g ON g.genre_id = t.genre_id
  WHERE g.name = 'Jazz'
),
customer_totals AS (
  SELECT
        customer_id, 
        SUM(total) AS total_spent
  FROM invoice
  GROUP BY customer_id
)
SELECT
    CASE
        WHEN ct.customer_id IN (SELECT customer_id FROM jazz_customers)
        THEN 'jazz' ELSE 'non_jazz'
    END AS customer_segment,
    AVG(ct.total_spent) AS avg_total_spent
FROM customer_totals ct
GROUP BY customer_segment;
"""

with engine.connect() as conn:
    df1 = pd.read_sql(query1, conn)
    print("Jazz vs Non-Jazz Customers Analysis:")
    print(df1)

Jazz vs Non-Jazz Customers Analysis:
  customer_segment  avg_total_spent
0             jazz        38.964375
1         non_jazz        40.064444


In [4]:
query2 = """
CREATE INDEX IF NOT EXISTS idx_track_lower_name
ON track (LOWER(name));
"""

try:
    with engine.begin() as conn:
        conn.execute(text(query2))
    print("Index created successfully!")
except Exception as e:
    print("Index creation failed:", e)

Index creation failed: (psycopg2.errors.InsufficientPrivilege) must be owner of table track

[SQL: 
CREATE INDEX IF NOT EXISTS idx_track_lower_name
ON track (LOWER(name));
]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [5]:
query3 = """
SELECT album.title
FROM track
JOIN album ON track.album_id = album.album_id
WHERE LOWER(track.name) = LOWER('Enter Sandman');
"""

with engine.connect() as conn:
    df3 = pd.read_sql(query3, conn)
    print("Search results for 'Enter Sandman':")
    print(df3)

Search results for 'Enter Sandman':
                            title
0  Plays Metallica By Four Cellos
1                     Black Album


In [6]:
# Close the connection when done
conn.close()
print("Connection closed.")

Connection closed.
